In [17]:
from haystack.nodes import FARMReader
from haystack.document_stores.faiss import FAISSDocumentStore
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.pipelines import ExtractiveQAPipeline

In [4]:
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2')

In [8]:
path = 'models/faiss'
faiss_document_store = FAISSDocumentStore.load(
    index_path = f'{path}/squad_dev.faiss',
    config_path= f'{path}/squad_dev.json'
)

In [9]:
es_document_store = ElasticsearchDocumentStore(
    host='localhost',
    index='squad_docs'
)

/Users/eliranboraks/opt/anaconda3/envs/haystack/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [15]:
retriever = DensePassageRetriever(
    document_store=es_document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    embed_title=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [16]:
es_document_store.update_embeddings(retriever=retriever)

Updating embeddings:   0%|          | 0/1204 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1216 [00:00<?, ? Docs/s]

In [18]:
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [21]:
pipeline.run(query="Who is U2")

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.50s/ Batches]


{'query': 'Who is U2',
 'no_ans_gap': -1.9333548545837402,
 'answers': [<Answer {'answer': 'Prohibition agent Eliot Ness, Supreme Court Justice John Paul Stevens, Prime Minister of Canada William Lyon Mackenzie King, 11th Prime Minister of Poland Marek Belka, Governor of the Bank of Japan Masaaki Shirakawa, the first female African-American Senator Carol Moseley Braun', 'type': 'extractive', 'score': 0.07054707407951355, 'context': 'Prohibition agent Eliot Ness, Supreme Court Justice John Paul Stevens, Prime Minister of Canada William Lyon Mackenzie King, 11th Prime Minister of Poland Marek Belka, Governor of the Bank of Japan Masaaki Shirakawa, the first female African-American Senator Carol Moseley Braun', 'offsets_in_document': [{'start': 289, 'end': 567}], 'offsets_in_context': [{'start': 0, 'end': 278}], 'document_id': '56ad9ca24c697e492c00e4f5b8bd9232', 'meta': {}}>,
  <Answer {'answer': 'prominent alumni include anthropologists David Graeber and Donald Johanson,', 'type': 'extra